In [ ]:
import socket
import threading

clients = []

def client_handler(conn, addr):
    print(f"[NEW CONNECTION] {addr} connected.")
    while True:
        try:
            msg = conn.recv(1024).decode()
            if not msg:
                break

            #close everything if exit received
            if msg.lower() == "exit":
                broadcast(f"{addr} has left the chat.", conn)
                conn.close()
                clients.remove(conn)
                break

            print(f"[{addr}] {msg}")
            broadcast(f"{addr}: {msg}", conn)

        except:
            conn.close()
            if conn in clients:
                clients.remove(conn)
            break

def broadcast(message, sender_conn):
    for client in clients:
        if client != sender_conn:
            try:
                client.send(message.encode())
            except:
                client.close()
                clients.remove(client)

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("127.0.0.1", 12345)) 
    server.listen()
    print("[SERVER STARTED] Listening on port 12345...")

    while True:
        conn, addr = server.accept()
        clients.append(conn)
        thread = threading.Thread(target=client_handler, args=(conn, addr))
        thread.start()

if __name__ == "__main__":
    start_server()


[SERVER STARTED] Listening on port 12345...
[NEW CONNECTION] ('127.0.0.1', 58552) connected.
[NEW CONNECTION] ('127.0.0.1', 58553) connected.
[('127.0.0.1', 58553)] hi
[('127.0.0.1', 58552)] hello
[('127.0.0.1', 58553)] how are you
[('127.0.0.1', 58552)] great, but currently busy
[('127.0.0.1', 58553)] ok will talk later then
